# File downloading

## Getting metadata from the API

In [1]:
import requests
import json

from tqdm.auto import tqdm

In [2]:
headers = {
"accept": "application/json, text/plain, */*",
"accept-encoding": "gzip, deflate, br",
"accept-language": "ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7",
"content-length": "215",
"content-type": "application/json;charset=UTF-8",
"origin": "https://www.hse.ru",
"referer": "https://www.hse.ru/edu/vkr/index.html?faculty=139191145&textAvailable=1&lang=ru",
"sec-fetch-dest": "empty",
"sec-fetch-mode": "cors",
"sec-fetch-site": "same-origin",
"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.183 Safari/537.36"
}

payload={"author":"",
         "vkrIds":"",
         "faculty":"139191145",
         "supervisor":"",
         "year":0,
         "title":"",
         "level":0,
         "score":0,
         "programs":"",
         "textAvailable":"1",
         "lang":"ru",
         "page": None,
         "findMode":False,
         "ruVersion":True,
         "urlPrefix":"/edu/vkr"}

def get_vkrs_from_page(headers, payload, page):
    
    payload["page"] = page

    url = 'https://www.hse.ru/edu/vkr/api/list'
    r = requests.post(url, data=json.dumps(payload), headers=headers)
    vkr_list = r.json()['result']['vkrs']['list']


    return vkr_list

In [4]:
vkrs = []

for page in tqdm(range(1,91)):
    
    vkrs.extend(get_vkrs_from_page(headers, payload, page))
    
print(len(vkrs))


895


In [5]:
vkrs[0]

{'supervisor_title': ['Кучерская Майя Александровна'],
 'supervisor_id': [135567],
 'supervisor_other': None,
 'en_supervisor_other': None,
 'ou_title': 'Факультет гуманитарных наук',
 'en_ou_title': 'Faculty of Humanities',
 'ou_id': 139191145,
 'level': 'Магистратура',
 'en_level': 'Master',
 'learn_program_title': 'Литературное мастерство',
 'en_learn_program_title': 'Creative Writing',
 'id': 370521098,
 'create_time': '2020-06-04T16:21:15.000Z',
 'code': '4B518870-0F70-4AC5-BE03-59824BCA4455',
 'title': 'Миллениалы',
 'last_name': 'Агеева',
 'first_name': 'Екатерина',
 'second_name': 'Анатольевна',
 'en_title': 'Millennials',
 'en_last_name': 'Ageeva',
 'en_first_name': 'Ekaterina',
 'en_second_name': 'Anatolevna',
 'rating': 8,
 'year': 2020,
 'text_available': 1,
 'type': 0,
 'learn_program_group': None,
 'learn_program': 267591984,
 'abstract': 'Роман в рассказах "Миллениалы" - это история о людях из раннего поколения Y, заставших развал СССР в детстве и переживавших 90-е гг. в

In [48]:
from collections import Counter

In [52]:
faculties = [v['learn_program_title'] for f in vkrs]
c = Counter(faculties)
programme_stat = dict(c.most_common())
programme_stat

{'История': 140,
 'Филология': 139,
 'Фундаментальная и компьютерная лингвистика': 119,
 'Философия': 98,
 'Культурология': 84,
 'Визуальная культура': 35,
 'История искусств': 34,
 'Философская антропология': 31,
 'Компьютерная лингвистика': 28,
 'Литературное мастерство': 25,
 'Прикладная культурология': 25,
 'История художественной культуры и рынок искусства': 23,
 'Теория языка и компьютерная лингвистика': 19,
 'Компаративистика: русская литература в кросс-культурной перспективе': 18,
 'Историческое знание': 13,
 'Языковая политика в условиях этнокультурного разнообразия': 10,
 'Философия и история религии': 10,
 'Русский как иностранный во взаимодействии языков и культур': 9,
 'Русская литература и компаративистика': 9,
 'Медиевистика': 7,
 'Культурная и интеллектуальная история: между Востоком и Западом': 5,
 'Философия и религиоведение': 5,
 'Лингвистическая теория и описание языка': 4,
 'История знания и социальная история': 3}

Writing to `.json` file

In [8]:
vkrs_dict = {vkr['vkr_id']: vkr for vkr in vkrs}
len(vkrs_dict)

895

In [9]:
with open ('vkrs_metada.json', 'w', encoding='utf-8') as output:
    json.dump(vkrs_dict, output, ensure_ascii=False, indent=2)

## Downloading texts

In [5]:
import os
import json
import requests

from tqdm.auto import tqdm

In [6]:
with open ('vkrs_metada.json', 'r', encoding='utf-8') as f:
    vkrs_dict = json.load(f)

In [8]:
def get_file_type(content_type):
    
    if content_type == 'application/vnd.openxmlformats-officedocument.wordprocessingml.document':
        file_type = 'docx'
    elif content_type == 'application/pdf':
        file_type = 'pdf'
    elif content_type == 'application/msword':
        file_type = 'doc'
    elif content_type == 'application/zip':
        file_type = 'zip'
    elif content_type == 'text/html; charset=UTF-8':
        file_type = 'html'
    elif content_type == 'application/vnd.ms-excel':
        file_type = 'xls'
    elif content_type == 'application/vnd.oasis.opendocument.text':
        file_type = 'odt'        
    else:
        file_type = 'unknown'
        
    return file_type

def download(vkr_id, vkr_file, filepath=''):
    resp = requests.get(vkr_file, allow_redirects=True)
    file_type = get_file_type(resp.headers.get('content-type'))
    
    if file_type in ['zip', 'xls', 'unknown', 'doc']:
        file_type = 'docx'
        
    file = f'{vkr_id}.{file_type}'
    
    full_path = os.path.join(filepath, file)
    open(full_path, 'wb').write(resp.content)

In [9]:
for v_id, v_item in tqdm(vkrs_dict.items()):
    try:
        download(v_id, v_item['vkr_file'], filepath='./raw_files')
    except Exception:
        print(v_id, v_item['vkr_file'])

153014193 http://lms.hse.ru/ap_service.php?getwork=1&guid=ED8FCFE9-D442-4C90-8F72-7B71D011D5F3
153011503 http://lms.hse.ru/ap_service.php?getwork=1&guid=D34F9F02-9F06-4E60-83BC-DD20161CEAA4
153008058 http://lms.hse.ru/ap_service.php?getwork=1&guid=C994AE14-DEBE-45C1-B1FE-494F178BE710
153007783 http://lms.hse.ru/ap_service.php?getwork=1&guid=87C840EC-D11F-4581-80CF-7EB03B0C0BCC
153014171 http://lms.hse.ru/ap_service.php?getwork=1&guid=D9514F20-32A7-4744-BCA1-C4FA3E72B7F6
153009893 http://lms.hse.ru/ap_service.php?getwork=1&guid=C05E4D56-91EE-4421-B075-D5C70CDF46A0
153006854 http://lms.hse.ru/ap_service.php?getwork=1&guid=A7AF35C6-EEE8-4989-B409-9C62C0DE9D17
153012578 http://lms.hse.ru/ap_service.php?getwork=1&guid=5F597686-4499-43C6-8E91-33541F5AC208
153007102 http://lms.hse.ru/ap_service.php?getwork=1&guid=A843AC79-ED70-43F7-AD27-B1B2FB5BA605
153014761 http://lms.hse.ru/ap_service.php?getwork=1&guid=E516325B-54FA-42D9-89D8-04EAD5E092F6
153016153 http://lms.hse.ru/ap_service.php?getwork

*Anna Polyanskaya, 2021*